In [75]:
import pandas as pd
import unicodedata
import re

fs_root = '/nearline/scicompsoft-public/flynp'
csv_file_path = 'fly-efish_metadata_2024-04-25.csv'
df = pd.read_csv(csv_file_path)
path_column_name = df.columns[0]

# Adapted from https://github.com/django/django/blob/main/django/utils/text.py
def slugify(value, allow_unicode=False):
    """
    Convert to ASCII if 'allow_unicode' is False. Convert spaces or repeated
    dashes to single dashes. Remove characters that aren't alphanumerics,
    underscores, or hyphens. Convert to lowercase. Also strip leading and
    trailing whitespace, dashes, and underscores.
    """
    value = str(value)
    if allow_unicode:
        value = unicodedata.normalize("NFKC", value)
    else:
        value = (
            unicodedata.normalize("NFKD", value)
            .encode("ascii", "ignore")
            .decode("ascii")
        )
    value = re.sub(r"[^\w\s-]", "", value.lower())
    return 'c_'+re.sub(r"[-\s]+", "_", value).strip("-_")

col2slug = {}
db_names = []
original_names = []
# Skip the first column which is the relpath
for original_name in df.columns[1:]:
    db_name = slugify(original_name)
    db_names.append(db_name)
    original_names.append(original_name)
    col2slug[original_name] = db_name

df_cols = pd.DataFrame(data={
    'db_name': db_names,
    'original_name': original_names
})
print(df_cols)

                db_name        original_name
0               c_batch                Batch
1               c_round                Round
2                c_date                 Date
3         c_batch_again          Batch again
4         c_round_again          Round again
5                 c_gel                  Gel
6              c_sample               Sample
7   c_driver_line_green  Driver line (green)
8       c_546_probe_red      546 probe (red)
9                 c_546                  546
10  c_647_probe_magenta  647 probe (magenta)
11                c_647                  647
12               c_zoom                 Zoom
13             c_region               Region
14           c_czi_or_b             czi or b
15           c_czi_if_b             czi if b


In [76]:
rename_logic = lambda x, idx: x if idx == 0 else col2slug[x]
df.columns = [rename_logic(x, i) for i, x in enumerate(df.columns)]
df.rename(columns={path_column_name: 'relpath'}, inplace=True)
df.insert(0, 'collection', fs_root)
df

,collection,relpath,c_batch,c_round,c_date,c_batch_again,c_round_again,c_gel,c_sample,c_driver_line_green,c_546_probe_red,c_546,c_647_probe_magenta,c_647,c_zoom,c_region,c_czi_or_b,c_czi_if_b
0,/nearline/scicompsoft-public/flynp,NP01_R1_20230906/NP01_R1_1_1_SS00790_AstA_546_...,NP01,R1,20230906,NP01,R1,1,1,SS00790,AstA,546,CCHa1,647,100x,LOL,czi,NaN
1,/nearline/scicompsoft-public/flynp,NP01_R1_20230906/NP01_R1_1_1_SS00790_AstA_546_...,NP01,R1,20230906,NP01,R1,1,1,SS00790,AstA,546,CCHa1,647,100x,ROL,czi,NaN
2,/nearline/scicompsoft-public/flynp,NP01_R1_20230906/NP01_R1_1_2_SS00328_AstA_546_...,NP01,R1,20230906,NP01,R1,1,2,SS00328,AstA,546,CCHa1,647,100x,LOL,czi,NaN
3,/nearline/scicompsoft-public/flynp,NP01_R1_20230906/NP01_R1_1_2_SS00328_AstA_546_...,NP01,R1,20230906,NP01,R1,1,2,SS00328,AstA,546,CCHa1,647,100x,ROL,czi,NaN
4,/nearline/scicompsoft-public/flynp,NP01_R1_20230906/NP01_R1_1_3_SS02594_AstA_546_...,NP01,R1,20230906,NP01,R1,1,3,SS02594,AstA,546,CCHa1,647,100x,LOL,czi,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2192,/nearline/scicompsoft-public/flynp,NP40_R2_20240308/NP40_R2_3_4_SS55960_CCHa1-CCH...,NP40,R2,20240308,NP40,R2,3,4,SS55960,CCHa1-CCHa2,546,SIFa-Hug,647,100x,Central,czi,NaN
2193,/nearline/scicompsoft-public/flynp,NP40_R2_20240308/NP40_R2_4_1_SS74396_Ms_546_Ca...,NP40,R2,20240308,NP40,R2,4,1,SS74396,Ms,546,Capa-Eh,647,100x,Central,czi,NaN
2194,/nearline/scicompsoft-public/flynp,NP40_R2_20240308/NP40_R2_4_2_SS72989_Ms_546_Ca...,NP40,R2,20240308,NP40,R2,4,2,SS72989,Ms,546,Capa-Eh,647,100x,Central,czi,NaN
2195,/nearline/scicompsoft-public/flynp,NP40_R2_20240308/NP40_R2_4_3_SS55961_Ms_546_Ca...,NP40,R2,20240308,NP40,R2,4,3,SS55961,Ms,546,Capa-Eh,647,080x,Central,czi,NaN


In [77]:
from sqlalchemy import create_engine

# Save the column map to the database
engine = create_engine('sqlite:///database.db')
df_cols.to_sql('metadata_columns', con=engine, if_exists='replace', index=False)

# Reconstruct the map from the database
query = "SELECT * FROM metadata_columns"
result_df = pd.read_sql_query(query, con=engine)
colname_map = {}
for index, row in result_df.iterrows():
    colname_map[row['db_name']] = row['original_name']
print(colname_map)


{'c_batch': 'Batch', 'c_round': 'Round', 'c_date': 'Date', 'c_batch_again': 'Batch again', 'c_round_again': 'Round again', 'c_gel': 'Gel', 'c_sample': 'Sample', 'c_driver_line_green': 'Driver line (green)', 'c_546_probe_red': '546 probe (red)', 'c_546': '546', 'c_647_probe_magenta': '647 probe (magenta)', 'c_647': '647', 'c_zoom': 'Zoom', 'c_region': 'Region', 'c_czi_or_b': 'czi or b', 'c_czi_if_b': 'czi if b'}


In [78]:
from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table

metadata = MetaData()
table_columns = [
    Column('id', Integer, primary_key=True),  # Autoincrements by default in many DBMS
    Column('collection', String, nullable=False),
    Column('relpath', String, nullable=False)
]
for colname in colname_map.keys():
    table_columns.append(Column(colname, String))

metadata_table = Table('metadata', metadata, *table_columns)
print(f"Dropping existing {metadata_table} table")
metadata_table.drop(engine, checkfirst=True)
metadata.create_all(engine)

df.to_sql(metadata_table.name, con=engine, if_exists='append', index=False)

# Write to disk
with engine.connect() as connection:
    connection.commit()

query = "SELECT * FROM metadata"
result_df = pd.read_sql_query(query, con=engine)
print(result_df)


Dropping existing metadata table
        id                          collection  \
0        1  /nearline/scicompsoft-public/flynp   
1        2  /nearline/scicompsoft-public/flynp   
2        3  /nearline/scicompsoft-public/flynp   
3        4  /nearline/scicompsoft-public/flynp   
4        5  /nearline/scicompsoft-public/flynp   
...    ...                                 ...   
2192  2193  /nearline/scicompsoft-public/flynp   
2193  2194  /nearline/scicompsoft-public/flynp   
2194  2195  /nearline/scicompsoft-public/flynp   
2195  2196  /nearline/scicompsoft-public/flynp   
2196  2197  /nearline/scicompsoft-public/flynp   

                                                relpath c_batch c_round  \
0     NP01_R1_20230906/NP01_R1_1_1_SS00790_AstA_546_...    NP01      R1   
1     NP01_R1_20230906/NP01_R1_1_1_SS00790_AstA_546_...    NP01      R1   
2     NP01_R1_20230906/NP01_R1_1_2_SS00328_AstA_546_...    NP01      R1   
3     NP01_R1_20230906/NP01_R1_1_2_SS00328_AstA_546_...    NP01   

In [79]:
import pandas as pd
from sqlalchemy import create_engine, text, inspect

# Path to the SQLite database
database_path = 'sqlite:///database.db'

# Create an SQLAlchemy engine
engine = create_engine(database_path)

# Function to get column names from a table using SQLAlchemy's reflection
def get_column_names(engine, table_name):
    # Use sqlalchemy.inspect() to get an inspector object
    inspector = inspect(engine)
    # Retrieve column information
    columns = inspector.get_columns(table_name)
    return [column['name'] for column in columns]

# Your table name
table_name = 'metadata'

# Get the column names
column_names = get_column_names(engine, table_name)

# Create the SQL query for searching "SS72989" across all columns
query_string = " OR ".join([f"{col} LIKE :search_string" for col in column_names])
full_query = text(f"SELECT * FROM {table_name} WHERE {query_string}")

search_string = '100x'

# Execute the query and load the results into a DataFrame
result_df = pd.read_sql_query(full_query, con=engine, params={'search_string': '%'+search_string+'%'})

print(result_df.shape)
# Display the DataFrame
result_df

(1110, 19)


,id,collection,relpath,c_batch,c_round,c_date,c_batch_again,c_round_again,c_gel,c_sample,c_driver_line_green,c_546_probe_red,c_546,c_647_probe_magenta,c_647,c_zoom,c_region,c_czi_or_b,c_czi_if_b
0,1,/nearline/scicompsoft-public/flynp,NP01_R1_20230906/NP01_R1_1_1_SS00790_AstA_546_...,NP01,R1,20230906,NP01,R1,1,1,SS00790,AstA,546,CCHa1,647,100x,LOL,czi,None
1,2,/nearline/scicompsoft-public/flynp,NP01_R1_20230906/NP01_R1_1_1_SS00790_AstA_546_...,NP01,R1,20230906,NP01,R1,1,1,SS00790,AstA,546,CCHa1,647,100x,ROL,czi,None
2,3,/nearline/scicompsoft-public/flynp,NP01_R1_20230906/NP01_R1_1_2_SS00328_AstA_546_...,NP01,R1,20230906,NP01,R1,1,2,SS00328,AstA,546,CCHa1,647,100x,LOL,czi,None
3,4,/nearline/scicompsoft-public/flynp,NP01_R1_20230906/NP01_R1_1_2_SS00328_AstA_546_...,NP01,R1,20230906,NP01,R1,1,2,SS00328,AstA,546,CCHa1,647,100x,ROL,czi,None
4,5,/nearline/scicompsoft-public/flynp,NP01_R1_20230906/NP01_R1_1_3_SS02594_AstA_546_...,NP01,R1,20230906,NP01,R1,1,3,SS02594,AstA,546,CCHa1,647,100x,LOL,czi,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1105,2192,/nearline/scicompsoft-public/flynp,NP40_R2_20240308/NP40_R2_3_3_SS55961_CCHa1-CCH...,NP40,R2,20240308,NP40,R2,3,3,SS55961,CCHa1-CCHa2,546,SIFa-Hug,647,100x,Central,czi,None
1106,2193,/nearline/scicompsoft-public/flynp,NP40_R2_20240308/NP40_R2_3_4_SS55960_CCHa1-CCH...,NP40,R2,20240308,NP40,R2,3,4,SS55960,CCHa1-CCHa2,546,SIFa-Hug,647,100x,Central,czi,None
1107,2194,/nearline/scicompsoft-public/flynp,NP40_R2_20240308/NP40_R2_4_1_SS74396_Ms_546_Ca...,NP40,R2,20240308,NP40,R2,4,1,SS74396,Ms,546,Capa-Eh,647,100x,Central,czi,None
1108,2195,/nearline/scicompsoft-public/flynp,NP40_R2_20240308/NP40_R2_4_2_SS72989_Ms_546_Ca...,NP40,R2,20240308,NP40,R2,4,2,SS72989,Ms,546,Capa-Eh,647,100x,Central,czi,None


In [80]:
# Get the metadata for a single image
import pandas as pd
from sqlalchemy import create_engine, text, inspect

engine = create_engine('sqlite:///database.db')
relpath = 'NP31_R2_20240119/NP31_R2_1_1_SS00090_Spab_546_Nplp1_647_100x_Central.zarr'
query = text(f"SELECT * FROM metadata WHERE relpath = :relpath")
result_df = pd.read_sql_query(query, con=engine, params={'relpath': relpath})
print(result_df)
metadata = result_df.iloc[0].to_dict()
for k in metadata.keys():
    if k == 'c_path': continue
    if k in colname_map:
        print(f"{colname_map[k]}: {metadata[k]}")


     id                          collection  \
0  1691  /nearline/scicompsoft-public/flynp   

                                             relpath c_batch c_round  \
0  NP31_R2_20240119/NP31_R2_1_1_SS00090_Spab_546_...    NP31      R2   

     c_date c_batch_again c_round_again c_gel c_sample c_driver_line_green  \
0  20240119          NP31            R2     1        1             SS00090   

  c_546_probe_red c_546 c_647_probe_magenta c_647 c_zoom c_region c_czi_or_b  \
0            Spab   546               Nplp1   647   100x  Central        czi   

  c_czi_if_b  
0       None  
Batch: NP31
Round: R2
Date: 20240119
Batch again: NP31
Round again: R2
Gel: 1
Sample: 1
Driver line (green): SS00090
546 probe (red): Spab
546: 546
647 probe (magenta): Nplp1
647: 647
Zoom: 100x
Region: Central
czi or b: czi
czi if b: None


In [81]:

import pandas as pd
from sqlalchemy import create_engine, text, Table, MetaData, func, select
from zarrcade.images import Image

image_id = "NP08_R2_20231017/NP08_R2_2_1_SS83741_Gpb5_546_ITP_647_100x_LOL.zarr"

metadata_table = Table('metadata', MetaData(), autoload_with=engine)
stmt = metadata_table.select().where(metadata_table.c.relpath == image_id)
print(stmt)



SELECT metadata.id, metadata.collection, metadata.relpath, metadata.c_batch, metadata.c_round, metadata.c_date, metadata.c_batch_again, metadata.c_round_again, metadata.c_gel, metadata.c_sample, metadata.c_driver_line_green, metadata.c_546_probe_red, metadata.c_546, metadata.c_647_probe_magenta, metadata.c_647, metadata.c_zoom, metadata.c_region, metadata.c_czi_or_b, metadata.c_czi_if_b 
FROM metadata 
WHERE metadata.relpath = :relpath_1


In [82]:

engine = create_engine('sqlite:///database.db')
meta = MetaData()
meta.reflect(bind=engine)



In [83]:
meta

MetaData()

In [85]:
meta.tables.get('images')

Table('images', MetaData(), Column('id', INTEGER(), table=<images>, primary_key=True, nullable=False), Column('collection', VARCHAR(), table=<images>, nullable=False), Column('relpath', VARCHAR(), table=<images>, nullable=False), Column('dataset', VARCHAR(), table=<images>, nullable=False), Column('image_path', VARCHAR(), table=<images>, nullable=False), Column('image_info', VARCHAR(), table=<images>, nullable=False), Column('metadata_id', INTEGER(), ForeignKey('metadata.id'), table=<images>), schema=None)

In [87]:
meta.tables.get('metadata2')

In [88]:
df

,collection,relpath,c_batch,c_round,c_date,c_batch_again,c_round_again,c_gel,c_sample,c_driver_line_green,c_546_probe_red,c_546,c_647_probe_magenta,c_647,c_zoom,c_region,c_czi_or_b,c_czi_if_b
0,/nearline/scicompsoft-public/flynp,NP01_R1_20230906/NP01_R1_1_1_SS00790_AstA_546_...,NP01,R1,20230906,NP01,R1,1,1,SS00790,AstA,546,CCHa1,647,100x,LOL,czi,NaN
1,/nearline/scicompsoft-public/flynp,NP01_R1_20230906/NP01_R1_1_1_SS00790_AstA_546_...,NP01,R1,20230906,NP01,R1,1,1,SS00790,AstA,546,CCHa1,647,100x,ROL,czi,NaN
2,/nearline/scicompsoft-public/flynp,NP01_R1_20230906/NP01_R1_1_2_SS00328_AstA_546_...,NP01,R1,20230906,NP01,R1,1,2,SS00328,AstA,546,CCHa1,647,100x,LOL,czi,NaN
3,/nearline/scicompsoft-public/flynp,NP01_R1_20230906/NP01_R1_1_2_SS00328_AstA_546_...,NP01,R1,20230906,NP01,R1,1,2,SS00328,AstA,546,CCHa1,647,100x,ROL,czi,NaN
4,/nearline/scicompsoft-public/flynp,NP01_R1_20230906/NP01_R1_1_3_SS02594_AstA_546_...,NP01,R1,20230906,NP01,R1,1,3,SS02594,AstA,546,CCHa1,647,100x,LOL,czi,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2192,/nearline/scicompsoft-public/flynp,NP40_R2_20240308/NP40_R2_3_4_SS55960_CCHa1-CCH...,NP40,R2,20240308,NP40,R2,3,4,SS55960,CCHa1-CCHa2,546,SIFa-Hug,647,100x,Central,czi,NaN
2193,/nearline/scicompsoft-public/flynp,NP40_R2_20240308/NP40_R2_4_1_SS74396_Ms_546_Ca...,NP40,R2,20240308,NP40,R2,4,1,SS74396,Ms,546,Capa-Eh,647,100x,Central,czi,NaN
2194,/nearline/scicompsoft-public/flynp,NP40_R2_20240308/NP40_R2_4_2_SS72989_Ms_546_Ca...,NP40,R2,20240308,NP40,R2,4,2,SS72989,Ms,546,Capa-Eh,647,100x,Central,czi,NaN
2195,/nearline/scicompsoft-public/flynp,NP40_R2_20240308/NP40_R2_4_3_SS55961_Ms_546_Ca...,NP40,R2,20240308,NP40,R2,4,3,SS55961,Ms,546,Capa-Eh,647,080x,Central,czi,NaN


In [99]:
from zarrcade.database import Database
from zarrcade.model import Image, MetadataImage

db = Database('sqlite:///database.db')

IMAGES_AND_METADATA_SQL = text("""
    SELECT m.*, i.image_path, i.image_info
    FROM 
        images i
    LEFT JOIN 
        metadata m
    ON 
        m.id = i.metadata_id
""")

import json
def deserialize_image_info(image_info: str):
    return Image(**json.loads(image_info))

def serialize_image_info(image: Image):
    return json.dumps(asdict(image))


def find_metaimages(search_string: str = '', page: int = 1, page_size: int = 10):
    """
    Find meta images with optional search and pagination.
    
    Args:
        search_string (str): The string to search for within image metadata.
        page (int): The page number (starting from 1) for pagination.
        page_size (int): The number of results per page.
    
    Returns:
        list: List of `MetadataImage` objects.
    """
    offset = (page - 1) * page_size

    base_query = IMAGES_AND_METADATA_SQL
    if not search_string:
        paginated_query = f"{base_query} LIMIT :limit OFFSET :offset"
        result_df = pd.read_sql_query(paginated_query, con=db.engine, params={'limit': page_size, 'offset': offset})
    else:
        cols = [f"m.{k}" for k in db.column_map.keys()] or ['i.relpath']
        query_string = " OR ".join([f"{col} LIKE :search_string" for col in cols])
        full_query = text(f"{base_query} WHERE {query_string} LIMIT :limit OFFSET :offset")
        result_df = pd.read_sql_query(full_query, con=db.engine, params={
            'search_string': f'%{search_string}%',
            'limit': page_size,
            'offset': offset
        })

    images = []
    for row in result_df.itertuples():
        metadata = db.get_tuple_metadata(row)
        image_info_json = row.image_info
        image_path = row.image_path
        if image_info_json:
            image = deserialize_image_info(image_info_json)
            metaimage = MetadataImage(image_path, image, metadata)
            images.append(metaimage)

    return images

images = find_metaimages(page=6, page_size=9)
len(images)

Registering column 'c_batch' for Batch
Registering column 'c_round' for Round
Registering column 'c_date' for Date
Registering column 'c_batch_again' for Batch again
Registering column 'c_round_again' for Round again
Registering column 'c_gel' for Gel
Registering column 'c_sample' for Sample
Registering column 'c_driver_line_green' for Driver line (green)
Registering column 'c_546_probe_red' for 546 probe (red)
Registering column 'c_546' for 546
Registering column 'c_647_probe_magenta' for 647 probe (magenta)
Registering column 'c_647' for 647
Registering column 'c_zoom' for Zoom
Registering column 'c_region' for Region
Registering column 'c_czi_or_b' for czi or b
Registering column 'c_czi_if_b' for czi if b


5

In [5]:


from sqlalchemy import create_engine, MetaData, Table, String
from sqlalchemy.orm import sessionmaker
from collections import defaultdict

def get_string_columns_with_few_unique_values(db_path, table_name, threshold=10):
    # Create the engine to connect to the SQLite database
    engine = create_engine(f'sqlite:///{db_path}')
    
    # Create a metadata instance
    metadata = MetaData()
    
    # Reflect the specified table from the database
    table = Table(table_name, metadata, autoload_with=engine)
    
    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()
    
    # Dictionary to store columns with less than the threshold unique values
    columns_with_few_unique_values = []
    
    try:
        # Iterate through columns to find string columns
        for column in table.columns:
            print(column, column.type)
            if isinstance(column.type, String):
                # Query to get the unique values count
                unique_values_count = session.query(column).distinct().count()
                
                if unique_values_count < threshold:
                    columns_with_few_unique_values.append(column.name)
                    
    finally:
        # Close the session
        session.close()
    
    return columns_with_few_unique_values

# Example usage:
db_path = '../database.db'
table_name = 'metadata'
print(get_string_columns_with_few_unique_values(db_path, table_name))


metadata.id INTEGER
metadata.collection VARCHAR
metadata.relpath VARCHAR
metadata.aux_image_path VARCHAR
metadata.thumbnail_path VARCHAR
metadata.c_batch VARCHAR
metadata.c_round VARCHAR
metadata.c_date VARCHAR
metadata.c_batch_again VARCHAR
metadata.c_round_again VARCHAR
metadata.c_gel VARCHAR
metadata.c_sample VARCHAR
metadata.c_driver_line_green VARCHAR
metadata.c_546_probe_red VARCHAR
metadata.c_647_probe_magenta VARCHAR
metadata.c_zoom VARCHAR
metadata.c_region VARCHAR
metadata.c_probes VARCHAR
['collection', 'c_batch', 'c_round', 'c_date', 'c_batch_again', 'c_round_again', 'c_gel', 'c_sample', 'c_546_probe_red', 'c_647_probe_magenta', 'c_zoom', 'c_region', 'c_probes']


In [11]:
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker

def get_unique_column_values(db_path, table_name, column_name):
    # Create the engine to connect to the SQLite database
    engine = create_engine(f'sqlite:///{db_path}')
    
    # Create a metadata instance
    metadata = MetaData()
    
    # Reflect the specified table from the database
    table = Table(table_name, metadata, autoload_with=engine)
    
    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()
    
    # Retrieve the column object
    column = table.c[column_name]
    
    # Get all unique values for the specified column
    unique_values = session.query(column).distinct().all()
    
    # Close the session
    session.close()
    
    # Extract values from tuples
    unique_values_list = [value[0] for value in unique_values]
    
    return unique_values_list

# Example usage:
db_path = '../database.db'
table_name = 'metadata'
column_name = 'c_driver_line_green'
print(column_name)
print(get_unique_column_values(db_path, table_name, column_name))
column_name = 'c_546_probe_red'
print(column_name)
print(get_unique_column_values(db_path, table_name, column_name))
column_name = 'c_647_probe_magenta'
print(column_name)
print(get_unique_column_values(db_path, table_name, column_name))
column_name = 'c_probes'
print(column_name)
print(get_unique_column_values(db_path, table_name, column_name))

c_driver_line_green
['SS00790', 'SS00328', 'SS02594', 'SS00308', 'SS00317', 'SS00809', 'SS00796', 'SS00321', 'SS01019', 'CCHa1-LexA', 'SS15156', 'SS21160', 'SS39717', 'SS03173', 'SS34042', 'SS61160', 'SS38292', 'SS02427', 'SS00323', 'SS00300', 'SS00320', 'SS00374', 'SS00698', 'SS02432', 'SS00384', 'SS03722', 'SS83741', 'SS00844', 'SS37203', 'SS47622', 'SS89446', 'SS00318', 'SS02353', 'SS00382', 'SS00778', 'SS00688', 'SS63542', 'SS77099', 'SS55882', 'SS61682', 'SS69117', 'SS63514', 'SS59709', 'SS02191', 'SS62862', 'SS82331', 'SS66112', 'SS59684', 'SS02916', 'SS65383', 'SS75731', 'SS02752', 'SS66004', 'SS60306', 'SS70711', 'SS56343', 'SS36564', 'SS56987', 'SS39313', 'SS81529', 'SS86882', 'SS15431', 'SS21658', 'SS35138', 'SS46630', 'SS76311', 'SS76386', 'SS77545', 'SS77582', 'SS78219', 'SS80761', 'SS80822', 'SS81667', 'SS84473', 'SS84487', 'SS88322', 'SS36363', 'SS87616', 'SS00657', 'MB122B', 'SS02394', 'SS14770', 'SS46115', 'SS87220', 'SS23921', 'SS05005', 'SS03188', 'SS34318', 'SS00298'

In [12]:
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker

def get_unique_comma_delimited_values(db_path, table_name, column_name):
    # Create the engine to connect to the SQLite database
    engine = create_engine(f'sqlite:///{db_path}')
    
    # Create a metadata instance
    metadata = MetaData()
    
    # Reflect the specified table from the database
    table = Table(table_name, metadata, autoload_with=engine)
    
    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()
    
    # Retrieve the column object
    column = table.c[column_name]
    
    # Get all values for the specified column
    values = session.query(column).all()
    
    # Close the session
    session.close()
    
    # Initialize a set to store unique values
    unique_values = set()
    
    # Process each row to split comma-delimited values and add to the set
    for value_tuple in values:
        value = value_tuple[0]
        if value:
            items = value.split(',')
            for item in items:
                unique_values.add(item.strip())
    
    # Convert the set to a sorted list
    unique_values_list = sorted(unique_values)
    
    return unique_values_list

# Example usage:
db_path = '../database.db'
table_name = 'metadata'
column_name = 'c_probes'
print(get_unique_comma_delimited_values(db_path, table_name, column_name))

['AstA', 'AstC', 'CCAP', 'CCAP-Eh', 'CCHa1', 'CCHa1-CCHa2', 'CCHa2', 'Capa', 'Capa-Eh', 'Crz', 'DH31', 'DH44', 'Eh', 'FMRFa', 'Gpb5', 'Hug', 'ITP', 'Lk-Eh', 'Mip', 'Ms', 'NPF', 'NPF-Trissin', 'Nplp1', 'Orcokinin', 'PROC', 'Proc', 'SIFA', 'SIFa', 'SIFa-Hug', 'Spab', 'TK', 'Tk', 'Trissin', 'no', 'sNPF']


In [3]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Define the SQLite database URL
DATABASE_URL = "sqlite:///example.db"

# Create an engine that connects to the SQLite database
engine = create_engine(DATABASE_URL)

# Create a base class for declarative models
Base = declarative_base()

# Define the metadata_columns table
class MetadataColumn(Base):
    __tablename__ = 'metadata_columns'

    id = Column(Integer, primary_key=True, autoincrement=True)
    db_name = Column(String, nullable=False, unique=True)
    original_name = Column(String, nullable=False)

# Create the table in the database if it doesn't exist
Base.metadata.create_all(engine)

# Create a session to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

def update_metadata_columns(data_dict):
    """
    Update the metadata_columns table with any entries missing from the given dictionary.
    
    Args:
        data_dict (dict): A dictionary where keys are db_name and values are original_name.
    """
    # Fetch existing entries
    existing_entries = session.query(MetadataColumn.db_name, MetadataColumn.original_name).all()
    existing_entries_dict = dict(existing_entries)

    # Prepare a list for new entries
    new_entries = []

    # Check for missing entries
    for db_name, original_name in data_dict.items():
        if db_name not in existing_entries_dict:
            new_entries.append(MetadataColumn(db_name=db_name, original_name=original_name))
        elif existing_entries_dict[db_name] != original_name:
            # Optionally update if original_name has changed
            existing_entry = session.query(MetadataColumn).filter_by(db_name=db_name).first()
            existing_entry.original_name = original_name
            session.add(existing_entry)
    
    # Add new entries to the session
    if new_entries:
        session.add_all(new_entries)
        session.commit()

# Example usage
data_dict = {
    "example_db1": "example_column1",
    "example_db2": "example_column2",
    "example_db3": "example_column3"
}

update_metadata_columns(data_dict)

/tmp/ipykernel_3308244/985640129.py:12: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [10]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import declarative_base, relationship
from sqlalchemy import create_engine, Column, Integer, String, Table, MetaData, Index
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import OperationalError

DATABASE_URL = "sqlite:///example.db"

# Create an engine that connects to the SQLite database
engine = create_engine(DATABASE_URL)

Base = declarative_base()

class Collection(Base):
    __tablename__ = 'collections'
    id = Column(Integer, primary_key=True)
    data_url = Column(String, nullable=False, unique=True)
    name = Column(String, nullable=False)


class MetadataColumn(Base):
    __tablename__ = 'metadata_columns'
    id = Column(Integer, primary_key=True, autoincrement=True)
    db_name = Column(String, nullable=False, unique=True)
    original_name = Column(String, nullable=False)
    

class ImageMetadata(Base):
    __tablename__ = 'image_metadata'    
    id = Column(Integer, primary_key=True)  # Autoincrements by default
    collection = Column(String, nullable=False)
    zarr_path = Column(String, nullable=False)
    aux_image_path = Column(String, nullable=True)
    thumbnail_path = Column(String, nullable=True)
    images = relationship('Image', back_populates='image_metadata', uselist=True)


class Image(Base):
    __tablename__ = 'images'
    id = Column(Integer, primary_key=True)
    collection = Column(String, nullable=False)
    zarr_path = Column(String, nullable=False)
    group_path = Column(String, nullable=False)
    image_path = Column(String, nullable=False, index=True)
    image_info = Column(String, nullable=False)
    image_metadata_id = Column(Integer, ForeignKey('image_metadata.id'), nullable=True, index=True)
    image_metadata = relationship('ImageMetadata', back_populates='images')

    # Define relationships if there is a Metadata class
    # 
    __table_args__ = (
        Index('collection_zarr_path_idx', 'collection', 'zarr_path'),
    )
    
Base.metadata.create_all(engine)